In [1]:
!pip install internetarchive

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 102 kB 10.7 MB/s           
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing wheel metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for internetarchive: filename=internetarchive-3.5.0-py3-none-any.whl size=95916 sha256=8679228476f7ba92bec48ffe6646606973f2775cf5f191cc28bba398b1d1e893
  Stored in directory: /home/jupyter-michael/.cache/pip/wheels/fa/13/e1/af00275618c05b9012e4afa5df9d38b8537a321a96ee010a64
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13724 sha256=8764daf6385ee3c1a2a6db3c1d04b997d47671b3fb82bc5e75329379c4832c7b
  Stored in directory: /home/jupyter-michael/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built internetarchive docopt
  Consider adding this directory to PATH or, if you prefer to 

In [7]:
import pandas as pd 
from uuid import uuid4
from datetime import datetime
from bs4 import BeautifulSoup
import internetarchive as ia

In [3]:
file_links = []

# Identifier of the Archive.org collection
collection_identifier = "schwurbel-archiv"

# Search for items in the collection
items = ia.search_items('collection:' + collection_identifier)


data = []

# Iterate over the items
for item in items:
    data.append({
        "ID": uuid4(),
        "identifier": item['identifier'],
        "status": "Added",
        "downloaded": "",
        "group_name": "",
        "crawl_data": ""
    })


In [5]:
import pandas as pd 

df = pd.DataFrame.from_dict(data)
df.to_csv('2023-07-06-Schwurbelarchiv-Liste.csv')

In [6]:
def update_status(index, status):
    df.at[index, 'status'] = status
    df.to_csv('2023-07-06-Schwurbelarchiv-Liste.csv', index=False)  # Save the DataFrame to a CSV file

In [8]:
def scrape_content(filepath):
  # Read the HTML file
  with open(filepath, 'r', encoding='utf-8') as file:
      html_content = file.read()

  # Create a Beautiful Soup object
  soup = BeautifulSoup(html_content, 'lxml')

  messages = soup.find_all('div', class_='message')
  data = []

  # Iterate over the messages
  for message_div in messages:
      from_name_div = message_div.find('div', class_='from_name')
      if not from_name_div:
          #print(message_div)
          continue

      from_name = from_name_div.get_text(strip=True)
      forwarded_div = message_div.find('div', class_='forwarded body')

      forwarded_from_name = ""
      forwarded_message_text = ""
      forwarded_message_photo = ""
      forwarded_video_duration = ""
      if forwarded_div:
          forwarded_from_name_div = forwarded_div.find('div', class_='from_name')
          forwarded_from_name = forwarded_from_name_div.get_text(strip=True).replace(forwarded_from_name_div.find('span', class_='details').get_text(strip=True), '')

          message_text_div = forwarded_div.find('div', class_='text')
          forwarded_message_text = ''.join(str(child) for child in message_text_div.contents) if message_text_div else ""

          message_photo = forwarded_div.find('img')
          forwarded_message_photo = message_photo['src'] if message_photo else ""

          video_duration_div = forwarded_div.find('div', class_='video_duration')
          forwarded_video_duration = video_duration_div.get_text(strip=True) if video_duration_div else ""


      message_text_div = message_div.find('div', class_='text')

      if message_text_div:
          message_text = ''.join(str(child) for child in message_text_div.contents) if message_text_div else ""

          message_photo = message_div.find('img')
          message_photo = message_photo['src'] if message_photo else ""

          video_duration_div = message_div.find('div', class_='video_duration')
          video_duration = video_duration_div.get_text(strip=True) if video_duration_div else ""

      date_element = message_div.find('div', class_='pull_right date details')
      datetime_value = date_element['title']
      datetime_object = datetime.strptime(datetime_value, '%d.%m.%Y %H:%M:%S')

      # Append the extracted data to the list
      data.append({
          'ID': uuid4(),
          'From Name': from_name,
          'Datetime': datetime_object,
          'Message Text': message_text,
          'Photo': message_photo,
          'Video Duration': video_duration,
          'Forwarded From Name': forwarded_from_name,
          'Forwarded Message Text': forwarded_message_text,
          'Forwarded Photo': forwarded_message_photo,
          'Forwarded Video Duration': forwarded_video_duration
      })

  # Create a pandas DataFrame from the extracted data
  return pd.DataFrame(data)

In [11]:
import zipfile
import os
import re
import shutil

In [ ]:
temp_folder_path = "tmp"

for index, row in df[11:20].iterrows():
    identifier = row['identifier']
    status = row['status']
    if status != 'Done':
        # Remove old files        
        if os.path.exists("export.zip"):
            os.remove("export*.zip")

        update_status(index, 'Downloading')

        item = ia.get_item(identifier)

        # Get group name
        name = item.metadata['title']

        # Extract the name
        name_pattern = r'„(.*?)“'
        name_match = re.search(name_pattern, name)
        group_name = name_match.group(1) if name_match else ""
        df.at[index, 'group_name'] = group_name

        # Extract the date
        date_pattern = r'vom (\d{2}\.\d{2}\.\d{4})'
        date_match = re.search(date_pattern, name)
        crawl_date = date_match.group(1) if date_match else ""
        # Convert the crawl_date_str to a datetime object
        crawl_date = datetime.strptime(crawl_date, "%d.%m.%Y")

        # Update the 'crawl_date' column in the DataFrame
        df.at[index, 'crawl_date'] = crawl_date

        print(f"Working on {identifier} -- {group_name}")

        download_files = []
        for f in ia.get_files(identifier):          
          if f.format == "ZIP":
            download_files.append(f)
            print(f"Downloading {f.name}")
            f.download()
        

        filename = ""
        if len(download_files) == 1:
          filename = "export.zip"

        else:
          filename = "export-part1.zip"

        update_status(index, 'Extracting')

        print("Downloaded. Extracting Now.")

        # Extract the export.zip into the temp folder
        with zipfile.ZipFile(filename, 'r') as zip_ref:
          zip_ref.extractall(temp_folder_path)

        update_status(index, 'Scraping')
        scraped_df = scrape_content(f"{temp_folder_path}/messages.html")
        scraped_df.to_csv(f"data/2023-07-Telegram-{row['ID']}.csv")

        # Remove the temp folder if it exists
        if os.path.exists(temp_folder_path):
          shutil.rmtree(temp_folder_path)

        if len(download_files) > 1:
          if os.path.exists(identifier):
            shutil.rmtree(identifier)

        update_status(index, 'Done')

print("Processing complete.")

Working on schwurbelarchiv-06hyV5GgkR -- Siegfried Daebritz
